# Lagrange Multipliers
*Last updated by Arthur Ryman on 2022-05-21*

## Introduction

The function `RWC_alam()` is currently failing for certain values of $B$.
I have tried using the SymPy functions `solve()`, `solveset()`, and `nsolve()` to find the optimal
parameters $a$ and $\lambda$ which parameterize the ground state, but the results are inconsistent.
Sometimes a solution is found, sometimes it fails.

The approach taken by Trevor is to find the minimum of the expectation value $E(a, \lambda)$, 
of the RWC Hamiltonian by constraining $\lambda$ to be a function of $a$ and solving for the zeros
of the total derivative of $E(a,\lambda(a))$ with respect to $a$.

Unfortunately, the resulting derivative is very complex and this may be defeating the SymPy algorithms.
Note that the corresponding Maple function `fsolve()` works consistently.

As a potential workaround, it might be easier to regard the problem as minimizing $E(a, \lambda)$ subject to
the constraint that relates $\lambda$ and $a$.
This type of problem can be solved using the method of *Lagrange multipliers*.

The goal of this notebook is to determine if the use of Lagrange multipliers improves the consistency of the
SymPy implementation of `RWC_alam()`.

## A Simpler Problem

I am going to first implement the approach using a very simple example.
Let the symbols $x, y$ be the coordinates.

Let $F(x,y)$ be the objective function and let $G(x,y)$ be the constraint.

In [1]:
from sympy import *

x = Symbol('x', real=True, positive=True)
y = Symbol('y', real=True, positive=True)

Let $F(x,y)$ be the objective function that we are going to minimize.

In [2]:
F = 4.5 * x + 3.2 * y
F

4.5*x + 3.2*y

Let $G(x,y)$ be the constraint that relates the coordinate.

In [3]:
G = x ** 2 + y ** 2 - 1
G

x**2 + y**2 - 1

In [4]:
dxF, dyF = F.diff(x), F.diff(y)
dxF

4.50000000000000

In [5]:
dyF

3.20000000000000

In [6]:
dxG, dyG = G.diff(x), G.diff(y)
dxG

2*x

In [7]:
dyG

2*y

Let $z$ be the Langrange multiplier.

In [8]:
z = Symbol('z', real=True)

We seek to find the solution of the following system of three equations.

In [9]:
system = dxF - z * dxG, dyF - z * dyG, G
system

(-2*x*z + 4.5, -2*y*z + 3.2, x**2 + y**2 - 1)

In [10]:
solutions = solve(system, [z, x, y], dict=True)

In [11]:
for s in solutions:
    print('solution: ', s)
    for e in system:
        print('equation: ', e, ', value: ', e.subs(s))

solution:  {z: 2.76088753845570, x: 0.814955324568755, y: 0.579523786360004}
equation:  -2*x*z + 4.5 , value:  0
equation:  -2*y*z + 3.2 , value:  0
equation:  x**2 + y**2 - 1 , value:  0


In [12]:
for s in solutions:
    print('solution: ', s)
    value = F.subs(s).evalf()
    print('F(x,y): ', value)

solution:  {z: 2.76088753845570, x: 0.814955324568755, y: 0.579523786360004}
F(x,y):  5.52177507691141


### Summary

The method worked well in this simple case. The function `solve()` returned a list of solutions.
Originally I did not define $x, y$ to be positive so two solutions were returned.

## The Function `RWC_alam()`

Now reimplement this function using Lagrange multipliers.

In [13]:
from acmpy.papers.wr2015 import *

Eq_61

Eq(lambda_v, lambda0 + v)

In [14]:
Eq_B11

Eq(lambda0, sqrt(a**4*beta0**4 + 9/4) + 1)

In [15]:
Eq_B15

Eq(beta0, Piecewise((sqrt(2)*sqrt(-c1/c2)/2, c1 < 0), (0, True)))

In [16]:
Eq_B16

Eq(E, B*c1*lambda0/(2*a**2) + B*c2*lambda0*(lambda0 + 1)/(2*a**4) + a**2*(1 + 9/(4*lambda0 - 4))/(2*B))

The code allows shifting $\lambda_0$ by $v$ via (61).
I assume the expectation value is given by (B.16) with $\lambda_v$ substituted for 
$\lambda_0$.

In [17]:
lambdav

lambda_v

In [18]:
Ev = E_B16(a, lambdav, B, c1, c2)
Ev

B*c1*lambda_v/(2*a**2) + B*c2*lambda_v*(lambda_v + 1)/(2*a**4) + a**2*(1 + 9/(4*lambda_v - 4))/(2*B)

Work with the variable $A = a^2$.

In [19]:
A = Symbol('A', real=True, positive=True)
EAv = Ev.subs(a ** 2, A)
EAv

A*(1 + 9/(4*lambda_v - 4))/(2*B) + B*c1*lambda_v/(2*A) + B*c2*lambda_v*(lambda_v + 1)/(2*A**2)

The code uses the variable $\mu$ instead of $\lambda$.

In [20]:
mu = Symbol('mu', real=True, positive=True)
mu

mu

In [21]:
def lambda_mu(mu):
    return 1 + mu / 2

lambda_mu(mu)

mu/2 + 1

In [22]:
EAmu = EAv.subs(lambdav, lambda_mu(mu))
EAmu

A*(1 + 9/(2*mu))/(2*B) + B*c1*(mu/2 + 1)/(2*A) + B*c2*(mu/2 + 1)*(mu/2 + 2)/(2*A**2)

The constraint expresses $\mu$ in terms of $A$.

In [23]:
from acmpy.hamiltonian_data import *

muf(A, c1, c2, v)

sqrt(A**2*c1**2/c2**2 + (2*v + 3)**2)

It may simplify the math if we square this expression.
Let $C(A, \mu) = 0$ be the constraint.

In [24]:
CAmu = mu ** 2 - muf(A, c1, c2, v) ** 2
CAmu

-A**2*c1**2/c2**2 + mu**2 - (2*v + 3)**2

With these changes, both the objective function `EAmu` and the constraint `CAmu`
are rational functions of $A$ and $\mu$.

In [25]:
d1E, d2E = EAmu.diff(A), EAmu.diff(mu)
d1E

(1 + 9/(2*mu))/(2*B) - B*c1*(mu/2 + 1)/(2*A**2) - B*c2*(mu/2 + 1)*(mu/2 + 2)/A**3

In [26]:
d2E

-9*A/(4*B*mu**2) + B*c1/(4*A) + B*c2*(mu/2 + 1)/(4*A**2) + B*c2*(mu/2 + 2)/(4*A**2)

In [27]:
d1C, d2C = CAmu.diff(A), CAmu.diff(mu)
d1C

-2*A*c1**2/c2**2

In [28]:
d2C

2*mu

In [29]:
E_system = d1E - z * d1C, d2E - z * d2C, CAmu
E_system

(2*A*c1**2*z/c2**2 + (1 + 9/(2*mu))/(2*B) - B*c1*(mu/2 + 1)/(2*A**2) - B*c2*(mu/2 + 1)*(mu/2 + 2)/A**3,
 -9*A/(4*B*mu**2) - 2*mu*z + B*c1/(4*A) + B*c2*(mu/2 + 1)/(4*A**2) + B*c2*(mu/2 + 2)/(4*A**2),
 -A**2*c1**2/c2**2 + mu**2 - (2*v + 3)**2)

In [30]:
# E_solutions = solve(E_system, [z, A, mu], dict=True)

Solving this system with symbolic parameters failed.
It took so long that I interrupted it before it finished.

In practice, we only need to solve it for numeric parameters.

In [31]:
def set_values(system: tuple[Expr],
               B_value: float, 
               c1_value: float, 
               c2_value: float, 
               v_value: int
              ) -> tuple[Expr]:
    values: dict = {B: B_value,
                   c1: c1_value,
                   c2: c2_value,
                   v: v_value}
    return tuple(t.subs(values).evalf() for t in system)

E_system_10 = set_values(E_system, 10, -3.0, 2.0, 0)
E_system_10

(4.5*A*z + 0.05 + 0.225/mu + 15.0*(0.5*mu + 1.0)/A**2 - 20.0*(0.5*mu + 1.0)*(0.5*mu + 2.0)/A**3,
 -0.225*A/mu**2 - 2.0*mu*z - 7.5/A + 5.0*(0.5*mu + 1.0)/A**2 + 5.0*(0.5*mu + 2.0)/A**2,
 -2.25*A**2 + mu**2 - 9.0)

In [32]:
# E_solutions_10 = solve(E_system_10, [z, A, mu], dict=True)

Using numeric values didn't help.

Try to solve the equations by solving for the Lagrange multiplier and then substituting.

In [33]:
E_system[0]

2*A*c1**2*z/c2**2 + (1 + 9/(2*mu))/(2*B) - B*c1*(mu/2 + 1)/(2*A**2) - B*c2*(mu/2 + 1)*(mu/2 + 2)/A**3

In [34]:
E_system[1]

-9*A/(4*B*mu**2) - 2*mu*z + B*c1/(4*A) + B*c2*(mu/2 + 1)/(4*A**2) + B*c2*(mu/2 + 2)/(4*A**2)

In [35]:
E_system[2]

-A**2*c1**2/c2**2 + mu**2 - (2*v + 3)**2

In [36]:
E_solution0 = solve(E_system[0], z)
len(E_solution0)

1

In [37]:
z_solution0 = E_solution0[0]
z_solution0

c2**2*(-2*A**3*mu - 9*A**3 + A*B**2*c1*mu*(mu + 2) + B**2*c2*mu*(mu**2 + 6*mu + 8))/(8*A**4*B*c1**2*mu)

In [38]:
E_system1Amu = E_system[1].subs(z, z_solution0)
E_system1Amu

-9*A/(4*B*mu**2) + B*c1/(4*A) + B*c2*(mu/2 + 1)/(4*A**2) + B*c2*(mu/2 + 2)/(4*A**2) - c2**2*(-2*A**3*mu - 9*A**3 + A*B**2*c1*mu*(mu + 2) + B**2*c2*mu*(mu**2 + 6*mu + 8))/(4*A**4*B*c1**2)

In [39]:
E_solution1 = solve(E_system1Amu, A)
len(E_solution1)

0

In [40]:
E_system1Amu_10 = set_values((E_system1Amu,), 10, -3.0, 2.0, 0)[0].expand()
E_system1Amu_10

-0.225*A/mu**2 + 0.0222222222222222*mu/A - 7.4/A + 5.0*mu/A**2 + 15.0/A**2 + 3.33333333333333*mu**2/A**3 + 6.66666666666667*mu/A**3 - 2.22222222222222*mu**3/A**4 - 13.3333333333333*mu**2/A**4 - 17.7777777777778*mu/A**4

In [41]:
solve(E_system1Amu_10, mu)

[]

### Summary

The equations are too complex to solve.

## Conclusion

The use of Lagrange multipliers did not help.

The goal is to find an optimal set of parameters, so we don't really need an exact solution.
Try using a SciPy optimizer.